In [ ]:
def hamming_encode(data):
    # Calculate the number of parity bits needed
    r = 0
    while 2**r < len(data) + r + 1:
        r += 1

    # Create the encoded data array with space for parity bits
    encoded_data = [0] * (len(data) + r)

    # Fill in the data bits
    j = 0
    for i in range(1, len(encoded_data) + 1):
        if i & (i - 1) == 0:
            continue  # Skip parity bits
        encoded_data[i - 1] = int(data[j])
        j += 1

    # Calculate parity bits
    for i in range(r):
        parity_index = 2**i - 1
        parity_value = 0
        for j in range(parity_index, len(encoded_data), 2 * parity_index + 2):
            parity_value ^= encoded_data[j]
        encoded_data[parity_index] = parity_value

    return ''.join(map(str, encoded_data))

def hamming_correct(encoded_data):
    # Calculate the number of parity bits needed
    r = 0
    while 2**r < len(encoded_data):
        r += 1

    # Initialize the error syndrome
    error_syndrome = 0

    # Calculate the error syndrome
    for i in range(r):
        parity_index = 2**i - 1
        parity_value = 0
        for j in range(parity_index, len(encoded_data), 2 * parity_index + 2):
            parity_value ^= int(encoded_data[j])
        error_syndrome |= (parity_value << i)

    # If the error syndrome is not zero, correct the error
    if error_syndrome != 0:
        print("Error detected at position:", error_syndrome)
        encoded_data = list(encoded_data)
        error_position = error_syndrome - 1
        encoded_data[error_position] = str(int(not int(encoded_data[error_position])))
        print("Corrected data:", ''.join(encoded_data))
    else:
        print("No errors detected. Data:", encoded_data)

def main():
    data = input("Enter the 4-bit data to be sent: ")

    # Encode the data
    encoded_data = hamming_encode(data)
    print("Encoded data:", encoded_data)

    # Introduce an error for demonstration purposes
    error_position = 5
    encoded_data = list(encoded_data)
    encoded_data[error_position - 1] = str(int(not int(encoded_data[error_position - 1])))
    print("Received data with error:", ''.join(encoded_data))

    # Correct the error
    hamming_correct(''.join(encoded_data))

if __name__ == "__main__":
    main()


Enter the 4-bit data to be sent: 1011
Encoded data: 0110011
Received data with error: 0110111
Error detected at position: 1
Corrected data: 1110111


In [ ]:
import sys

class Graph:
    def __init__(self, vertices):
        self.vertices = vertices
        self.graph = [[0] * vertices for _ in range(vertices)]

    def add_edge(self, u, v, weight):
        self.graph[u][v] = weight
        self.graph[v][u] = weight

    def dijkstra(self, start):
        distance = [sys.maxsize] * self.vertices
        distance[start] = 0
        visited = [False] * self.vertices

        for _ in range(self.vertices):
            min_distance = sys.maxsize
            min_index = 0

            for v in range(self.vertices):
                if distance[v] < min_distance and not visited[v]:
                    min_distance = distance[v]
                    min_index = v

            visited[min_index] = True

            for v in range(self.vertices):
                if (
                    self.graph[min_index][v] > 0
                    and not visited[v]
                    and distance[v] > distance[min_index] + self.graph[min_index][v]
                ):
                    distance[v] = distance[min_index] + self.graph[min_index][v]

        self.print_solution(distance)

    def print_solution(self, distance):
        print("Vertex \t Distance from Source")
        for node in range(self.vertices):
            print(f"{node}\t {distance[node]}")


# Example usage:
if __name__ == "__main__":
    graph = Graph(6)
    graph.add_edge(0, 1, 4)
    graph.add_edge(0, 2, 2)
    graph.add_edge(1, 2, 5)
    graph.add_edge(1, 3, 10)
    graph.add_edge(2, 3, 3)
    graph.add_edge(2, 4, 2)
    graph.add_edge(3, 4, 7)
    graph.add_edge(4, 5, 1)

    start_vertex = 0
    print(f"Shortest paths from vertex {start_vertex} using Dijkstra's algorithm:")
    graph.dijkstra(start_vertex)


Shortest paths from vertex 0 using Dijkstra's algorithm:
Vertex 	 Distance from Source
0	 0
1	 4
2	 2
3	 5
4	 4
5	 5


In [ ]:
import sys

class DistanceVectorRouter:
    def __init__(self, nodes):
        self.nodes = nodes
        self.routing_table = {node: {dest: sys.maxsize for dest in range(nodes)} for node in range(nodes)}

    def add_link(self, node1, node2, cost):
        self.routing_table[node1][node2] = cost
        self.routing_table[node2][node1] = cost

    def update_routing_table(self):
        for node in range(self.nodes):
            for dest in range(self.nodes):
                if dest != node:
                    for neighbor in range(self.nodes):
                        if (
                            self.routing_table[node][neighbor] + self.routing_table[neighbor][dest]
                            < self.routing_table[node][dest]
                        ):
                            self.routing_table[node][dest] = self.routing_table[node][neighbor] + self.routing_table[neighbor][dest]

    def print_routing_table(self):
        print("Routing Table:")
        for node in range(self.nodes):
            print(f"Node {node}: {self.routing_table[node]}")

def main():
    # Create a router with 5 nodes
    router = DistanceVectorRouter(5)

    # Add links with associated costs
    router.add_link(0, 1, 1)
    router.add_link(0, 2, 3)
    router.add_link(1, 3, 4)
    router.add_link(2, 3, 2)
    router.add_link(2, 4, 5)
    router.add_link(3, 4, 1)

    # Initial routing table
    router.print_routing_table()

    # Simulate updates until convergence
    for _ in range(3):
        router.update_routing_table()
        router.print_routing_table()

if __name__ == "__main__":
    main()


Routing Table:
Node 0: {0: 9223372036854775807, 1: 1, 2: 3, 3: 9223372036854775807, 4: 9223372036854775807}
Node 1: {0: 1, 1: 9223372036854775807, 2: 9223372036854775807, 3: 4, 4: 9223372036854775807}
Node 2: {0: 3, 1: 9223372036854775807, 2: 9223372036854775807, 3: 2, 4: 5}
Node 3: {0: 9223372036854775807, 1: 4, 2: 2, 3: 9223372036854775807, 4: 1}
Node 4: {0: 9223372036854775807, 1: 9223372036854775807, 2: 5, 3: 1, 4: 9223372036854775807}
Routing Table:
Node 0: {0: 9223372036854775807, 1: 1, 2: 3, 3: 5, 4: 6}
Node 1: {0: 1, 1: 9223372036854775807, 2: 4, 3: 4, 4: 5}
Node 2: {0: 3, 1: 4, 2: 9223372036854775807, 3: 2, 4: 3}
Node 3: {0: 5, 1: 4, 2: 2, 3: 9223372036854775807, 4: 1}
Node 4: {0: 6, 1: 5, 2: 3, 3: 1, 4: 9223372036854775807}
Routing Table:
Node 0: {0: 9223372036854775807, 1: 1, 2: 3, 3: 5, 4: 6}
Node 1: {0: 1, 1: 9223372036854775807, 2: 4, 3: 4, 4: 5}
Node 2: {0: 3, 1: 4, 2: 9223372036854775807, 3: 2, 4: 3}
Node 3: {0: 5, 1: 4, 2: 2, 3: 9223372036854775807, 4: 1}
Node 4: {0: 6

In [ ]:
import sys

class LinkStateRouter:
    def __init__(self, nodes):
        self.nodes = nodes
        self.link_state_database = {node: {dest: sys.maxsize for dest in range(nodes)} for node in range(nodes)}

    def add_link(self, node1, node2, cost):
        self.link_state_database[node1][node2] = cost
        self.link_state_database[node2][node1] = cost

    def dijkstra(self, start):
        distance = {node: sys.maxsize for node in range(self.nodes)}
        distance[start] = 0
        visited = set()

        while len(visited) < self.nodes:
            current = min((node for node in range(self.nodes) if node not in visited), key=lambda x: distance[x])
            visited.add(current)

            for neighbor in range(self.nodes):
                if neighbor not in visited and self.link_state_database[current][neighbor] > 0:
                    new_distance = distance[current] + self.link_state_database[current][neighbor]
                    if new_distance < distance[neighbor]:
                        distance[neighbor] = new_distance

        return distance

    def print_link_state_database(self):
        print("Link State Database:")
        for node in range(self.nodes):
            print(f"Node {node}: {self.link_state_database[node]}")

def main():
    # Create a router with 5 nodes
    router = LinkStateRouter(5)

    # Add links with associated costs
    router.add_link(0, 1, 1)
    router.add_link(0, 2, 3)
    router.add_link(1, 3, 4)
    router.add_link(2, 3, 2)
    router.add_link(2, 4, 5)
    router.add_link(3, 4, 1)

    # Initial link state database
    router.print_link_state_database()

    # Calculate shortest paths using Dijkstra's algorithm
    for node in range(5):
        shortest_paths = router.dijkstra(node)
        print(f"Shortest paths from Node {node}: {shortest_paths}")

if __name__ == "__main__":
    main()


Link State Database:
Node 0: {0: 9223372036854775807, 1: 1, 2: 3, 3: 9223372036854775807, 4: 9223372036854775807}
Node 1: {0: 1, 1: 9223372036854775807, 2: 9223372036854775807, 3: 4, 4: 9223372036854775807}
Node 2: {0: 3, 1: 9223372036854775807, 2: 9223372036854775807, 3: 2, 4: 5}
Node 3: {0: 9223372036854775807, 1: 4, 2: 2, 3: 9223372036854775807, 4: 1}
Node 4: {0: 9223372036854775807, 1: 9223372036854775807, 2: 5, 3: 1, 4: 9223372036854775807}
Shortest paths from Node 0: {0: 0, 1: 1, 2: 3, 3: 5, 4: 6}
Shortest paths from Node 1: {0: 1, 1: 0, 2: 4, 3: 4, 4: 5}
Shortest paths from Node 2: {0: 3, 1: 4, 2: 0, 3: 2, 4: 3}
Shortest paths from Node 3: {0: 5, 1: 4, 2: 2, 3: 0, 4: 1}
Shortest paths from Node 4: {0: 6, 1: 5, 2: 3, 3: 1, 4: 0}


In [ ]:
def get_ip_class(ip_address):
    first_octet = int(ip_address.split('.')[0])
    if 1 <= first_octet <= 126:
        return 'A'
    elif 128 <= first_octet <= 191:
        return 'B'
    elif 192 <= first_octet <= 223:
        return 'C'
    elif 224 <= first_octet <= 239:
        return 'D (Multicast)'
    elif 240 <= first_octet <= 255:
        return 'E (Reserved)'
    else:
        return 'Invalid IP address'

def generate_subnet(ip_address, subnet_mask):
    ip_parts = list(map(int, ip_address.split('.')))
    mask_parts = list(map(int, subnet_mask.split('.')))

    subnet = []
    for i in range(4):
        subnet.append(ip_parts[i] & mask_parts[i])

    return '.'.join(map(str, subnet))

def main():
    ip_address = input("Enter the IP address: ")

    ip_class = get_ip_class(ip_address)
    print(f"The IP address {ip_address} belongs to Class {ip_class}")

    subnet_mask = input("Enter the subnet mask: ")

    subnet_ip = generate_subnet(ip_address, subnet_mask)
    print(f"The subnet IP address is: {subnet_ip}")

if __name__ == "__main__":
    main()


Enter the IP address: 192.168.1.6
The IP address 192.168.1.6 belongs to Class C
Enter the subnet mask: 255.255.255.0
The subnet IP address is: 192.168.1.0


In [ ]:
import numpy as np

# Define the gene expression data
gene_names = ['Gene1', 'Gene2', 'Gene3', 'Gene4', 'Gene5']
features = np.array([[0.1, 0.2, 0.3, 0.4, 0.5],
                     [0.6, 0.7, 0.8, 0.9, 1.0],
                     [0.2, 0.4, 0.6, 0.8, 1.0],
                     [0.3, 0.5, 0.7, 0.9, 1.1],
                     [0.4, 0.6, 0.8, 1.0, 1.2]])
labels = np.array([2.5, 3.2, 1.8, 2.4, 3.1])

def linear_regression(X, y):
    n_samples, n_features = X.shape

    # Calculate the mean of each feature
    mean_x = np.mean(X, axis=0)

    # Center the data by subtracting the mean of each feature
    X_centered = X - mean_x

    # Calculate the covariance matrix
    covariance_matrix = np.dot(X_centered.T, X_centered) / (n_samples - 1)

    # Calculate the weight vector
    w = np.dot(np.linalg.inv(covariance_matrix), np.dot(X_centered.T, y))

    # Make predictions
    y_pred = np.dot(X, w) + mean_x @ w

    return w, y_pred

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2)

# Train the linear regression model
w, y_pred = linear_regression(X_train, y_train)

# Evaluate the model's performance
mean_squared_error = np.mean((y_test - y_pred)**2)
print('MSE:', mean_squared_error)

# Print the coefficients of the model
print('Coefficients:', w)


NameError: ignored

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

# Define the gene expression data
gene_names = ['Gene1', 'Gene2', 'Gene3', 'Gene4', 'Gene5']
features = np.array([[0.1, 0.2, 0.3, 0.4, 0.5],
                     [0.6, 0.7, 0.8, 0.9, 1.0],
                     [0.2, 0.4, 0.6, 0.8, 1.0],
                     [0.3, 0.5, 0.7, 0.9, 1.1],
                     [0.4, 0.6, 0.8, 1.0, 1.2]])
labels = np.array([2.5, 3.2, 1.8, 2.4, 3.1])

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2)

# Train the linear regression model
model = LinearRegression()
model.fit(X_train, y_train)

# Make predictions
y_pred = model.predict(X_test)

# Evaluate the model's performance
from sklearn.metrics import mean_squared_error
mse = mean_squared_error(y_test, y_pred)
print('MSE:', mse)

# Print the coefficients of the model
print('Coefficients:', model.coef_)


MSE: 6.5024999999999995
Coefficients: [ 6.63333333  3.96666667  1.3        -1.36666667 -4.03333333]


In [ ]:
import numpy as np

# Define the population genetics data
allele_frequency = np.array([0.2, 0.4, 0.3, 0.5, 0.6])
environmental_factor = np.array([10, 20, 30, 40, 50])
trait_value = np.array([15, 22, 27, 34, 41])

# Combine features into a single matrix
features = np.column_stack((allele_frequency, environmental_factor))

# Split the data into training and testing sets using slicing
n_samples = len(features)
n_train = int(0.8 * n_samples)

X_train = features[:n_train]
y_train = trait_value[:n_train]

X_test = features[n_train:]
y_test = trait_value[n_train:]

def linear_regression(features, labels):
    n_samples, n_features = features.shape

    # Calculate the mean of each feature
    mean_x = np.mean(features, axis=0)

    # Center the data by subtracting the mean of each feature
    X_centered = features - mean_x

    # Calculate the covariance matrix
    covariance_matrix = np.dot(X_centered.T, X_centered) / (n_samples - 1)

    # Calculate the weight vector
    w = np.dot(np.linalg.inv(covariance_matrix), np.dot(X_centered.T, labels))

    # Make predictions
    y_pred = np.dot(features, w) + mean_x @ w

    return w, y_pred

# Train the linear regression model
w, y_pred = linear_regression(X_train, y_train)

# Evaluate the model's performance
mean_squared_error = np.mean((y_test - y_pred)**2)
print('MSE:', mean_squared_error)

# Print the coefficients of the model
print('Coefficients:', w)


MSE: 3798.249999999988
Coefficients: [20.   1.7]
